In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from math import nan

pd.set_option('display.max_columns', None)


# Aufbereitung

## Monatsbericht


In [2]:
def prepare_monatsbericht(path_origin, path_aim):
    df_1082 = pd.read_csv(path_origin, skiprows =7,
            skipfooter = 4, sep = ';'
            )
    #Adapt column names
    df_1082.columns = ['Steller', 'Produkt', 'Betriebe', 'Betriebe_ep', 'Beschäftigte',
       'Beschäftigte_ep', 'Geleistete Arbeitsstunden', 'Geleistete Arbeitsstunden_ep',
       'Bruttolohn- und -gehaltssumme', 'Bruttolohn- und -gehaltssumme_ep', 'Umsatz', 'Umsatz_ep',
       'Inlandsumsatz', 'Inlandsumsatz_ep', 'Auslandsumsatz', 'Auslandsumsatz_ep',
       'Auslandsumsatz mit der Eurozone', 'Auslandsumsatz mit der Eurozone_ep',
       'Auslandsumsatz mit dem sonstigen Ausland', 'Auslandsumsatz mit dem sonstigen Ausland_ep']
    different_steller = df_1082['Steller'].unique().tolist()
    num_diff_steller = len(different_steller)-12-7-1
    #Add date
    df_1082['month']=0
    for p in range(7):
        k = 2 + (1+(num_diff_steller +1)*12)*p
        for month in range(1,13): 
            for i in range(1,(num_diff_steller +1)):
                df_1082.iloc[k+i+(month-1)*(num_diff_steller +1), -1] = month
    for p in range(7):
        k = 2 + (1+(num_diff_steller +1)*12)*p
        for month in range(0,12): 
            df_1082 =  df_1082.drop(k + month * (num_diff_steller +1),axis=0)
    df_1082['year']=0
    for k in range(7):
        for i in range(1 + 12 * num_diff_steller):
            df_1082.iloc[1 + k* (1 + 12 * num_diff_steller) + i, -1] = k + 2019
    for k in range(7):
        df_1082 =df_1082.drop(1 + k*(1 + 12 * (num_diff_steller+1)),axis = 0)
    df_1082 =df_1082.drop(0,axis = 0)
    df_1082['day']=1
    df_1082['date']=pd.to_datetime(df_1082[['year', 'month', 'day']])

    df_1082 = df_1082.drop('month', axis = 1)
    df_1082 = df_1082.drop('year', axis = 1)
    df_1082 = df_1082.drop('day', axis = 1)
    #Include multiplicities
    for col in ['Betriebe', 'Beschäftigte', 'Geleistete Arbeitsstunden', 'Bruttolohn- und -gehaltssumme', 
            'Inlandsumsatz', 'Auslandsumsatz', 'Auslandsumsatz mit der Eurozone', 'Umsatz',
            'Auslandsumsatz mit dem sonstigen Ausland']:
        df_1082[col]=df_1082[col].replace('...', nan)
        df_1082[col]=df_1082[col].astype('float64')
    df_1082['Geleistete Arbeitsstunden']=df_1082['Geleistete Arbeitsstunden'] * 1000
    df_1082['Bruttolohn- und -gehaltssumme']=df_1082['Bruttolohn- und -gehaltssumme'] * 1000
    df_1082['Umsatz']=df_1082['Umsatz'] * 1000
    df_1082['Inlandsumsatz']=df_1082['Inlandsumsatz'] * 1000
    df_1082['Auslandsumsatz']=df_1082['Auslandsumsatz'] * 1000
    df_1082['Auslandsumsatz mit der Eurozone']=df_1082['Auslandsumsatz mit der Eurozone'] * 1000
    df_1082['Auslandsumsatz mit dem sonstigen Ausland']=df_1082['Auslandsumsatz mit dem sonstigen Ausland'] * 1000
    #Divide df in a df for each different Steller
    different_steller = df_1082['Steller'].unique().tolist()
    df_separated = []
    for steller in different_steller:
        mask = (df_1082['Steller'] == steller)
        df_help = df_1082.loc[mask,:]
        df_help = df_help.drop('Steller', axis=1)
        new_columns = []
        for col in df_help.columns.values[:-1]:
            new_columns.append(col + '_' + steller)
        new_columns.append('date')
        df_help.columns = new_columns
        df_help = df_help.set_index('date')
        df_separated.append(df_help)
    df_1082_prepared = pd.concat(df_separated, axis = 1)
    df_1082_prepared.to_csv(path_aim)

In [3]:
path_origin = '../data/raw/1082ff_monatsbericht_suesswaren_schoko_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_monatsbericht(path_origin, path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\2423498605.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df_1082 = pd.read_csv(path_origin, skiprows =7,


In [4]:
path_origin = '../data/raw/1082_monatsbericht_suesswaren_timeseries.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_monatsbericht(path_origin, path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\2423498605.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df_1082 = pd.read_csv(path_origin, skiprows =7,


## Einfuhr und Erzeugerpreise

In [5]:
def prepare_GP(path_origin, path_aim, praefix = ''):
    df= pd.read_csv(path_origin, 
                skiprows =6,
                skipfooter = 3, sep = ';'
                )
    for i in range(3, 171, 2):
        zahl = str(i)
        df = df.drop('Unnamed: '+zahl, axis=1)
    df = df.transpose()
    columns = df.iloc[1,:]
    columns.values[0] = 'Month'
    for i in range(1,len(columns)):
        columns.values[i] = praefix + '_' + columns.values[i]
    df.columns = columns
    df = df.drop('Unnamed: 0', axis = 0)
    df = df.drop('Unnamed: 1', axis = 0)
    list_year = []
    for i in range(7):
        list_year = list_year + np.full(12,i+2019).tolist()
    df['Year'] = list_year
    df['Day']=1
    df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5, 
                                   'Juni': 6, 'Juli': 7, 'August': 8, 'September': 9, 'Oktober': 10, 'November': 11, 'Dezember': 12})

    df['date']=pd.to_datetime(df[['Year', 'Month', 'Day']])
    df = df.drop('Month', axis = 1)
    df = df.drop('Year', axis = 1)
    df = df.drop('Day', axis = 1)
    df = df.set_index('date')
    df.to_csv(path_aim)

In [6]:
path_origin = '../data/raw/1082ff_erzeugerpreise_suesswaren_schoko_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_GP(path_origin, path_aim, 'ErzPr')

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\99283346.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
C:\Users\judit\AppData\Local\Temp\ipykernel_30068\99283346.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5,


In [7]:
path_origin = '../data/raw/1082_1083_einfuhrpreise_suesswarenundkaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_GP(path_origin, path_aim, 'EinfPr')

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\99283346.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
C:\Users\judit\AppData\Local\Temp\ipykernel_30068\99283346.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5,


### VPI

In [10]:
path_origin = '../data/raw/01183ff_vpi_suesswaren_schoko_kaffee.csv'

In [43]:
df= pd.read_csv(path_origin, 
                skiprows =6,
                skipfooter = 4, sep = ';'
                )
for i in range(3, 171, 2):
    zahl = str(i)
    df = df.drop('Unnamed: '+zahl, axis=1)
df = df.transpose()
praefix = 'VPI'
columns = df.iloc[1,:]
columns.values[1] = 'Month'
for i in range(2,len(columns)):
    columns.values[i] = praefix + '_' + columns.values[i]
df.columns = columns
df = df.drop('Verbraucherpreisindex (2020=100)', axis = 0)
df = df.drop('Unnamed: 1', axis = 0)
df = df.drop(nan, axis = 1)
list_year = []
for i in range(7):
    list_year = list_year + np.full(12,i+2019).tolist()
df['Year'] = list_year
df['Day']=1
df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5, 
                                   'Juni': 6, 'Juli': 7, 'August': 8, 'September': 9, 'Oktober': 10, 'November': 11, 'Dezember': 12})

df['date']=pd.to_datetime(df[['Year', 'Month', 'Day']])
df = df.drop('Month', axis = 1)
df = df.drop('Year', axis = 1)
df = df.drop('Day', axis = 1)
df = df.set_index('date')
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
df.to_csv(path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\1381331265.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
C:\Users\judit\AppData\Local\Temp\ipykernel_30068\1381331265.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5,


### A1014


In [130]:
path_origin = '../data/raw/A1014_17_18_monprod_suesswarenequi_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'

In [131]:
df= pd.read_csv(path_origin, 
                skiprows =4,
                skipfooter = 4, sep = ';'
                )
list_month = [nan, nan]
for i in range(1,13):
    list_month = list_month + np.full(22,i).tolist()
df.loc[0,:]=list_month
list_year = [nan, nan, nan]
for i in range(2019,2026):
    list_year = list_year + np.full(4,i).tolist()
df.loc[:,'Deutschland']=list_year
for i in range(7):
    df = df.drop(3+ i *4)
df.loc[1, 'Deutschland'] = 1
df.loc[1, 'Unnamed: 1'] = 1
# Ab hier monatsspezifisch
df_monate = []
for i in range(1,13):
    df_help = df.loc[:, (df.loc[0, :] == i) | (df.loc[1,:] == 1)]
    df_help['Month'] = i
    df_help = df_help.drop(0)
    columns = ['Year', 'Name', 'Wert der zum Absatz bestimmten Produktion', 
           'Wert der zum Absatz bestimmten Produktion_ep', 
           'Gewicht der zum Absatz bestimmten Produktion_t',
           'Gewicht der zum Absatz bestimmten Produktion_t_ep',
           'Gewicht der zum Absatz bestimmten Produktion_kg', 
           'Gewicht der zum Absatz bestimmten Produktion_kg_ep', 
           'Menge der zum Absatz bestimmten Produktion_stueck', 
           'Menge der zum Absatz bestimmten Produktion_stueck_ep', 
           'Menge der zum Absatz bestimmten Produktion_jewME', 
           'Menge der zum Absatz bestimmten Produktion_jewME_ep',
           'Unternehmen mit zum Absatz bestimmter Produktion',
           'Unternehmen mit zum Absatz bestimmter Produktion_ep',
           'Gewicht der Gesamtproduktion_t',
           'Gewicht der Gesamtproduktion_t_ep', 
           'Gewicht der Gesamtproduktion_kg',
           'Gewicht der Gesamtproduktion_kg_ep', 
           'Menge der Gesamtproduktion_stueck', 
           'Menge der Gesamtproduktion_stueck_ep', 
           'Menge der Gesamtproduktion_jewME',
           'Menge der Gesamtproduktion_jewME_ep', 
           'Unternehmen mit Produktion',
           'Unternehmen mit Produktion_ep', 
           'Month'
           ]
    df_help.columns = columns
    df_help = df_help.drop(1)
    vielfachheiten = df_help.loc[2, :]
    df_help = df_help.drop(2)
    for col in ['Wert der zum Absatz bestimmten Produktion', 
           'Gewicht der zum Absatz bestimmten Produktion_t',
           'Gewicht der zum Absatz bestimmten Produktion_kg', 
           'Menge der zum Absatz bestimmten Produktion_stueck',  
           'Menge der zum Absatz bestimmten Produktion_jewME', 
           'Unternehmen mit zum Absatz bestimmter Produktion',
           'Gewicht der Gesamtproduktion_t',
           'Gewicht der Gesamtproduktion_kg',
           'Menge der Gesamtproduktion_stueck', 
           'Menge der Gesamtproduktion_jewME',
           'Unternehmen mit Produktion']:
        df_help[col]=df_help[col].replace('...', nan)
        df_help[col]=df_help[col].replace('-', nan)
        df_help[col]=df_help[col].astype('float64')
    df_help['Year']=df_help['Year'].astype('int64')
    df_help['Month']=df_help['Month'].astype('int64')
    df_help['Wert der zum Absatz bestimmten Produktion']=df_help['Wert der zum Absatz bestimmten Produktion']*1000
    df_help['Day']=1
    df_help['date']=pd.to_datetime(df_help[['Year', 'Month', 'Day']])

    df_help = df_help.drop('Month', axis = 1)
    df_help = df_help.drop('Year', axis = 1)
    df_help = df_help.drop('Day', axis = 1)
    df_help = df_help.set_index('date')
    df_monate.append(df_help)

df_end = pd.concat(df_monate, axis = 0)
different_steller = df_end['Name'].unique().tolist()
df_separated = []
for steller in different_steller:
    mask = (df_end['Name'] == steller)
    df_help = df_end.loc[mask,:]
    df_help = df_help.drop('Name', axis=1)
    new_columns = []
    for col in df_help.columns.values:
        new_columns.append(col + '_' + steller)
    #new_columns.append('date')
    df_help.columns = new_columns
    #df_help = df_help.set_index('date')
    df_separated.append(df_help)
df_endagain = pd.concat(df_separated, axis = 1)
df_endagain.to_csv(path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\1850794607.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
C:\Users\judit\AppData\Local\Temp\ipykernel_30068\1850794607.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_help['Month'] = i
C:\Users\judit\AppData\Local\Temp\ipykernel_30068\1850794607.py:63: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_help[col]=df_

In [132]:
df_endagain

,"Wert der zum Absatz bestimmten Produktion_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Wert der zum Absatz bestimmten Produktion_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Gewicht der zum Absatz bestimmten Produktion_t_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Gewicht der zum Absatz bestimmten Produktion_t_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Gewicht der zum Absatz bestimmten Produktion_kg_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Gewicht der zum Absatz bestimmten Produktion_kg_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Menge der zum Absatz bestimmten Produktion_stueck_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Menge der zum Absatz bestimmten Produktion_stueck_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Menge der zum Absatz bestimmten Produktion_jewME_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Menge der zum Absatz bestimmten Produktion_jewME_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Unternehmen mit zum Absatz bestimmter Produktion_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Unternehmen mit zum Absatz bestimmter Produktion_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Gewicht der Gesamtproduktion_t_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Gewicht der Gesamtproduktion_t_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Gewicht der Gesamtproduktion_kg_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Gewicht der Gesamtproduktion_kg_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Menge der Gesamtproduktion_stueck_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Menge der Gesamtproduktion_stueck_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Menge der Gesamtproduktion_jewME_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Menge der Gesamtproduktion_jewME_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Unternehmen mit Produktion_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln","Unternehmen mit Produktion_ep_Leb-, Honigkuchen, Kekse, Kleingebäck, Waffeln",Wert der zum Absatz bestimmten Produktion_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Wert der zum Absatz bestimmten Produktion_ep_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Gewicht der zum Absatz bestimmten Produktion_t_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Gewicht der zum Absatz bestimmten Produktion_t_ep_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Gewicht der zum Absatz bestimmten Produktion_kg_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Gewicht der zum Absatz bestimmten Produktion_kg_ep_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Menge der zum Absatz bestimmten Produktion_stueck_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Menge der zum Absatz bestimmten Produktion_stueck_ep_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Menge der zum Absatz bestimmten Produktion_jewME_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Menge der zum Absatz bestimmten Produktion_jewME_ep_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Unternehmen mit zum Absatz bestimmter Produktion_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Unternehmen mit zum Absatz bestimmter Produktion_ep_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Gewicht der Gesamtproduktion_t_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Gewicht der Gesamtproduktion_t_ep_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Gewicht der Gesamtproduktion_kg_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Gewicht der Gesamtproduktion_kg_ep_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Menge der Gesamtproduktion_stueck_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Menge der Gesamtproduktion_stueck_ep_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Menge der Gesamtproduktion_jewME_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Menge der Gesamtproduktion_jewME_ep_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Unternehmen mit Produktion_Schokolade u.a. kakaohaltige Lebensmittelzubereit.,Unternehmen mit Produktion_ep_Schokolade u.a. 

### A1017


In [135]:
path_origin = '../data/raw/A1017_monprod_timeseries.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'


In [136]:
df= pd.read_csv(path_origin, 
                skiprows =4,
                skipfooter = 4, sep = ';'
                )
df

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\2126497088.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,


,Deutschland,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Unnamed: 146,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Unnamed: 151,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175,Unnamed: 176,Unnamed: 177,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184,Unnamed: 185,Unnamed: 186,Unnamed: 187,Unnamed: 188,Unnamed: 189,Unnamed: 190,Unnamed: 191,Unnamed: 192,Unnamed: 193,Unnamed: 194,Unnamed: 195,Unnamed: 196,Unnamed: 197,Unnamed: 198,Unnamed: 199,Unnamed: 200,Unnamed: 201,Unnamed: 202,Unnamed: 203,Unnamed: 204,Unnamed: 205,Unnamed: 206,Unnamed: 207,Unnamed: 208,Unnamed: 209,Unnamed: 210,Unnamed: 211,Unnamed: 212,Unnamed: 213,Unnamed: 214,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,Unnamed: 221,Unnamed: 222,Unnamed: 223,Unnamed: 224,Unnamed: 225,Unnamed: 226,Unnamed: 227,Unnamed: 228,Unnamed: 229,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239,Unnamed: 240,Unnamed: 241,Unnamed: 242,Unnamed: 243,Unnamed: 244,Unnamed: 245,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255,Unnamed: 256,Unnamed: 257,Unnamed: 258,Unnamed: 259,Unnamed: 260,Unnamed: 261,Unnamed: 262,Unnamed: 263,Unnamed: 264,Unnamed: 265
0,NaN,NaN,Januar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Februar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,März,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,April,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juni,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juli,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,August,NaN,N

### globalcacaoprice


In [ ]:
path_origin = '../data/raw/globalcacaoprice_fred.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'


### globalcoffeerobustaprice


In [ ]:
path_origin = '../data/raw/globalcoffeerobustaprice_fred.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'


### 0118ff waegung vpi

In [ ]:
path_origin = '../data/0118ff_waegung_vpi_suesswaren_schoko_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
